Chi-square test and post hoc

ref: https://neuhofmo.github.io/chi-square-and-post-hoc-in-python/

In [1]:
# imports
import pandas as pd

from scipy.stats import chi2_contingency
from statsmodels.sandbox.stats.multicomp import multipletests # for multiple comparisons correction
from itertools import combinations  # for post-hoc tests

In [2]:
# assists in displaying significance
def get_asterisks_for_pval(p_val, alpha=0.05):
    """Receives the p-value and returns asterisks string."""
    if p_val > alpha:  # bigger than alpha
        p_text = "ns"
    # following the standards in biological publications
    elif p_val < 1e-4:  
        p_text = '****'
    elif p_val < 1e-3:
        p_text = '***'
    elif p_val < 1e-2:
        p_text = '**'
    else:
        p_text = '*'
    
    return p_text  # string of asterisks

In [3]:
def run_chisq_on_combination(df, combinations_tuple):
    """Receives a dataframe and a combinations tuple and returns p-value after performing chisq test."""
    assert len(combinations_tuple) == 2, "Combinations tuple is too long! Should be of size 2."
    new_df = df[(df.index == combinations_tuple[0]) | (df.index == combinations_tuple[1])]
    chi2, p, dof, ex = chi2_contingency(new_df, correction=True)
    return p


def chisq_and_posthoc_corrected(df, correction_method='fdr_bh', alpha=0.05):
    """Receives a dataframe and performs chi2 test and then post hoc.
    Prints the p-values and corrected p-values (after FDR correction).
    alpha: optional threshold for rejection (default: 0.05)
    correction_method: method used for mutiple comparisons correction. (default: 'fdr_bh').
    See statsmodels.sandbox.stats.multicomp.multipletests for elaboration."""

    # start by running chi2 test on the matrix
    chi2, p, dof, ex = chi2_contingency(df, correction=True)
    print("Chi2 result of the contingency table: {}, p-value: {}\n".format(chi2, p))
    
    # post-hoc test
    all_combinations = list(combinations(df.index, 2))  # gathering all combinations for post-hoc chi2
    print("Post-hoc chi2 tests results:")
    p_vals = [run_chisq_on_combination(df, comb) for comb in all_combinations]  # a list of all p-values
    # the list is in the same order of all_combinations

    # correction for multiple testing
    reject_list, corrected_p_vals = multipletests(p_vals, method=correction_method, alpha=alpha)[:2]
    for p_val, corr_p_val, reject, comb in zip(p_vals, corrected_p_vals, reject_list, all_combinations):
        print("{}: p_value: {:5f}; corrected: {:5f} ({}) reject: {}".format(comb, p_val, corr_p_val, get_asterisks_for_pval(p_val, alpha), reject))

Applying the function on my data

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.getcwd()

Mounted at /content/gdrive


'/content'

In [5]:
# loading the file from excel
df = pd.read_excel('/content/gdrive/MyDrive/DataColab/groups_sum_crosstab.xlsx', index_col='treatment')

In [6]:
df

,died3h,Died3h7d,Lived
treatment,,,
NoDREADD,8.0,3.0,20.0
hM4Di,3.0,5.0,4.0
hM3Dq,0.0,0.0,14.0


In [20]:
df.iloc[0,2] = 20

In [21]:
df

,died3h,Died3h7d,Lived
treatment,,,
NoDREADD,8.0,3.0,20.0
hM4Di,3.0,5.0,4.0
hM3Dq,0.0,0.0,14.0


In [22]:
chisq_and_posthoc_corrected(df)


Chi2 result of the contingency table: 16.851447947214076, p-value: 0.002065709704501387

Post-hoc chi2 tests results:
('NoDREADD', 'hM4Di'): p_value: 0.043548; corrected: 0.043548 (*) reject: True
('NoDREADD', 'hM3Dq'): p_value: 0.037348; corrected: 0.043548 (*) reject: True
('hM4Di', 'hM3Dq'): p_value: 0.001182; corrected: 0.003545 (**) reject: True
